## METHOD 1 : Using Beautiful Soup

In [1]:
# importing the libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
#Lets define the URL

url="https://www.worldometers.info/coronavirus/"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse HTML code for the entire site
soup = BeautifulSoup(html_content, "lxml")
#print(soup.prettify()) # print the parsed data of html

In [3]:
#we pick the id of the table we want to scrape and extract HTML code for that particular table only
covid_table = soup.find("table", attrs={"id": "main_table_countries_today"})

In [1]:
#the head will form our columns
head = covid_table.thead.find_all("tr") 
head #the headers are contained in this HTML code

NameError: name 'covid_table' is not defined

In [5]:
headings = []
for th in head[0].find_all("th"):
    # remove any newlines and extra spaces from left and right
    print(th.text)
    #headings.append(td.b.text.replace('\n', ' ').strip())
    headings.append(th.text.replace("\n","").strip())
print(headings)

#
Country,Other
TotalCases
NewCases
TotalDeaths
NewDeaths
TotalRecovered
NewRecovered
ActiveCases
Serious,Critical
Tot Cases/1M pop
Deaths/1M pop
TotalTests
Tests/
1M pop

Population
Continent
1 Caseevery X ppl
1 Deathevery X ppl
1 Testevery X ppl
['#', 'Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical', 'Tot\xa0Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop', 'Population', 'Continent', '1 Caseevery X ppl', '1 Deathevery X ppl', '1 Testevery X ppl']


In [7]:
body = covid_table.tbody.find_all("tr") 
body[0] #here is one example of HTML snippet for one row

<tr class="total_row_world row_continent" data-continent="North America" style="display: none">
<td></td>
<td style="text-align:left;">
<nobr>North America</nobr>
</td>
<td>3,217,018</td>
<td>+38,937</td>
<td>170,062</td>
<td>+1,050</td>
<td>1,400,097</td>
<td>+8,110</td>
<td>1,646,859</td>
<td>18,867</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td data-continent="North America" style="display:none;">North America</td>
<td>
</td>
<td></td>
<td></td>
</tr>

In [8]:
#lets declare empty list data that will hold all rows data
data = []
for r in range(1,len(body)):
    row = [] # empty lsit to hold one row data
    for tr in body[r].find_all("td"):
        row.append(tr.text.replace("\n","").strip())
        #append row data to row after removing newlines escape and triming unnecesary spaces
    data.append(row)
    
# data contains all the rows excluding header
# row contains data for one row

In [11]:
#We can now pass data into a pandas dataframe
#with headings as the columns
df = pd.DataFrame(data,columns=headings)
df.head(10)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,1,USA,"2,758,395","+30,542","130,446",+324,"1,146,785","+3,451","1,481,164","15,801","8,333",394,"34,473,682","104,149","331,002,277",North America,120,"2,537",10
1,2,Brazil,"1,426,913","+18,428","60,194",+538,"790,040",,"576,679","8,318","6,713",283,"3,070,447","14,445","212,558,178",South America,149,"3,531",69
2,3,Russia,"654,405","+6,556","9,536",+216,"422,931","+10,281","221,938","2,300","4,484",65,"19,852,167","136,035","145,934,619",Europe,223,"15,304",7
3,4,India,"604,808","+19,016","17,848",+438,"359,891","+12,055","227,069","8,944",438,13,"8,826,585","6,396","1,379,974,505",Asia,"2,282","77,318",156
4,5,UK,"313,483",+829,"43,906",+176,N/A,N/A,N/A,238,"4,618",647,"9,662,051","142,327","67,886,052",Europe,217,"1,546",7
5,6,Spain,"296,739",+388,"28,363",+8,N/A,N/A,N/A,617,"6,347",607,"5,448,984","116,544","46,754,824",Europe,158,"1,648",9
6,7,Peru,"285,213",,"9,677",,"174,535",,"101,001","1,185","8,651",294,"1,679,386","50,937","32,969,875",South America,116,"3,407",20
7,8,Chile,"282,043","+2,650","5,753",+65,"245,443","+4,214","30,847","2,106","14,754",301,"1,109,792","58,056","19,115,944",South America,68,"3,323",17
8,9,Italy,"240,760",+182,"34,788",+21,"190,717",+469,"15,255",87,"3,982",575,"5,445,476","90,065","60,461,520",Europe,251,"1,738",11
9,10,Iran,"230,211","+2,549","10,958",+141,"191,487","+2,729","27,766","3,081","2,741",130,"1,693,242","20,160","83,988,944",Asia,365,"7,665",50


In [10]:
data = df[df["#"]!=""].reset_index(drop=True)
# Data points with # value are the countries of the world while the data points with
# null values for # columns are features like continents totals etc
data = data.drop_duplicates(subset = ["Country,Other"])
#Reason to drop duplicates : Worldometer reports data for 3 days: today and 2 days back
#I found out that removing duplicates removes the values for the bast two days and keep today's

In [237]:
#We can drop the following columns - Opinion
cols = ['#',
 'Tot\xa0Cases/1M pop',
 'Deaths/1M pop',
 'Tests/1M pop',
 'Population',
 '1 Caseevery X ppl',
 '1 Deathevery X ppl',
 '1 Testevery X ppl']

In [238]:
data_final = data.drop(cols,axis=1)
data_final.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotalTests,Continent
0,USA,"2,711,956","+30,145","129,977",+556,"1,126,736","+9,559","1,455,243","15,930","33,582,461",North America
1,Brazil,"1,383,678","+13,190","58,927",+542,"757,462",,"567,289","8,318","3,070,447",South America
2,Russia,"647,849","+6,693","9,320",+154,"412,650","+9,220","225,879","2,300","19,562,440",Europe
3,India,"585,792","+18,256","17,410",+506,"347,836","+12,565","220,546","8,944","8,608,654",Asia
4,UK,"312,654",+689,"43,730",+155,N/A,N/A,N/A,276,"9,426,631",Europe


In [243]:
#Once you are happy then you can save the dataframe as csv or xlsx
#data_final.to_csv("covid.csv",index=False)

In [242]:
#Here is a list of continents
list(data_final["Continent"].unique())

['North America',
 'South America',
 'Europe',
 'Asia',
 'Africa',
 'Australia/Oceania',
 '']

In [244]:
#Then you can filter by continent
data_final[data_final["Continent"]=="Africa"];

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotalTests,Continent
16,South Africa,"151,209","+6,945","2,657",+128,"73,543","+2,929","75,009",539,"1,630,008",Africa
22,Egypt,"68,311","+1,557","2,953",+81,"18,460",+509,"46,898",41,"135,000",Africa
48,Nigeria,"25,133",,573,,"9,402",,"15,158",7,"132,304",Africa
53,Ghana,"17,741",+390,112,,"13,268",+274,"4,361",6,"294,867",Africa
58,Algeria,"13,907",+336,912,+7,"9,897",+223,"3,098",42,,Africa
62,Cameroon,"12,592",,313,,"10,100",,"2,179",52,,Africa
63,Morocco,"12,533",+243,228,+3,"8,920",+87,"3,385",7,"681,191",Africa
65,Ivory Coast,"9,499",+285,68,+2,"4,273",+277,"5,158",,"55,950",Africa
66,Sudan,"9,257",,572,,"4,014",,"4,671",,401,Africa
72,DRC,"7,039",+100,170,+3,"1,426",+376,"5,443",,,Africa


## METHOD 2 : USING XPath

In [21]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np

In [23]:
#Rememeber that rows on HTML are stored between <tr>..</tr> tags
url = "https://www.worldometers.info/coronavirus/"
page = requests.get(url)#Store the contents of the website under doc
doc = lh.fromstring(page.content)#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr') #note that we are only using tr because we have only one table on the site
#if we had many then we could have needed to have the XPath so that we are specific.
#XPath can be accessed by inspecting elements of a site. 
print(len(list(tr_elements))) #number of all rows including the header row

696


In [24]:
#Lets deal with the header row first
col=[]
i=0#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))


1:"#"
2:"Country,Other"
3:"TotalCases"
4:"NewCases"
5:"TotalDeaths"
6:"NewDeaths"
7:"TotalRecovered"
8:"NewRecovered"
9:"ActiveCases"
10:"Serious,Critical"
11:"Tot Cases/1M pop"
12:"Deaths/1M pop"
13:"TotalTests"
14:"Tests/
1M pop
"
15:"Population"
16:"Continent"
17:"1 Caseevery X ppl"
18:"1 Deathevery X ppl"
19:"1 Testevery X ppl"


In [25]:
# Since out first row is the header, data is stored on the second row onwards
for j in range(1, len(tr_elements)):
    # T is our j'th row
    T = tr_elements[j]

    # If row is not of size 10, the //tr data is not from our table
    if len(T) != 19:
        break

    # i is the index of our column
    i = 0

    # Iterate through each element of the row
    for t in T.iterchildren():
        data = t.text_content()
        # Check if row is empty
        if i > 0:
            # Convert any numerical value to integers
            try:
                data = int(data)
            except:
                pass
        # Append the data to the empty list of the i'th column
        col[i][1].append(data)
        # Increment i for the next column
        i += 1

#ideally all columns must have the same number of rows
print([len(C) for (title,C) in col]) #for this case we have 695

[695, 695, 695, 695, 695, 695, 695, 695, 695, 695, 695, 695, 695, 695, 695, 695, 695, 695, 695]


In [26]:
#From here the same explanation holds like in method 1 above
Dict = {title:column for (title,column) in col}
df = pd.DataFrame(Dict)
data = df[df["#"]!=""].reset_index(drop=True)
data = data.drop_duplicates(subset = ["Country,Other"])

In [27]:
cols = ['#',
 'Tot\xa0Cases/1M pop',
 'Deaths/1M pop',
 'Tests/\n1M pop\n',
 'Population',
 'Continent',
 '1 Caseevery X ppl',
 '1 Deathevery X ppl',
 '1 Testevery X ppl']

data_final = data.drop(cols,axis=1)
#Assume we want to pick only these 5 countries
c = ["South Africa" , "Rwanda","Senegal","Ghana","Cameroon"]
data = data_final[data_final["Country,Other"].isin(c)].reset_index(drop=True)
data.head(10)

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotalTests
0,South Africa,"159,333","+8,124","2,749",92,"76,025","+2,482","80,559",539,"1,666,939"
1,Ghana,"17,741",,112,,"13,268",,"4,361",6,"297,591"
2,Cameroon,"12,592",,313,,"10,100",,"2,179",52,
3,Senegal,"6,925",132,116,4,"4,545",114,"2,264",35,"80,619"
4,Rwanda,"1,025",,2,,447,,576,,"143,943"
